In [1]:
from circuit import gen_random_circuits
from circuit.formatter import qiskit_to_my_format_circuit, layered_circuits_to_qiskit
from upstream import RandomwalkModel
from collections import defaultdict

import math
from utils.backend import gen_grid_topology, get_grid_neighbor_info, Backend, gen_linear_topology, get_linear_neighbor_info
# from utils.backend import default_basis_single_gates, default_basis_two_gates

from qiskit import QuantumCircuit
import pennylane as qml
import numpy as np
from downstream.synthesis.pennylane_op import layer_circuit_to_pennylane_circuit, layer_circuit_to_pennylane_tape
from downstream.synthesis.tensor_network_op import layer_circuit_to_matrix
from downstream.synthesis.synthesis_model_dnn import matrix_distance_squared

from scipy.stats import unitary_group

from downstream.synthesis.synthesis_model_pca_unitary import pkl_dump, pkl_load
from downstream.synthesis.synthesis_model_pca_unitary import SynthesisModel

import time
from qiskit import transpile
import random
from itertools import combinations

# 2-qubit topological information
n_qubits = 4
topology = gen_linear_topology(n_qubits)
neigh_info = get_linear_neighbor_info(n_qubits, 1)


backend = Backend(n_qubits=n_qubits, topology=topology, neighbor_info=neigh_info, basis_single_gates=['u'],
                  basis_two_gates=['cz'], divide=False, decoupling=False)

model_name = f'q{n_qubits}_02204_dnn_unitary_eval_data'

In [2]:
# '''生成用于测试的模板电路'''
# max_gate = 2**n_qubits
# min_gate = 5
# circuits = gen_random_circuits(min_gate=min_gate, max_gate=max_gate, gate_num_step=max(
#     [1, (max_gate+max_gate)//50]), n_circuits=50, two_qubit_gate_probs=[3, 8], backend=backend, reverse=False, optimize=True, multi_process=True)

# '''生成一些一些比特之间没有纠缠的'''
# all_qubits = list(range(n_qubits))
# for i in range(len(all_qubits) + 1):
#     for subset in combinations(all_qubits, i):
#         if len(subset) <= 1 or len(subset) == n_qubits:
#             continue
#         n_qubit_subset = len(subset)
#         max_gate = 4**(n_qubit_subset+1)
#         print(subset, n_qubit_subset, max_gate)
#         sub_backend = backend.get_sub_backend(subset)
        
#         if len(sub_backend.coupling_map) == 0:
#             continue
        
#         circuits += gen_random_circuits(min_gate=min_gate, max_gate=max_gate, gate_num_step=max([1, (max_gate+max_gate)//50]), n_circuits=5, 
#                                        two_qubit_gate_probs=[3, 8], backend=sub_backend, reverse=False, optimize=True, multi_process=False)

# upstream_model = RandomwalkModel(
#     1, 20, backend=backend, travel_directions=('parallel', 'next'))
# upstream_model.train(circuits, multi_process=True)

In [3]:
# len(circuits)

In [4]:
# random.shuffle(upstream_model.dataset)
# circuits = upstream_model.dataset
# synthesis_model = SynthesisModel(
#     upstream_model=upstream_model, name=model_name)
# synthesis_model.construct_data(circuits=circuits, multi_process=True)
# synthesis_model.save()
# print('finish construct data')

In [5]:
synthesis_model: SynthesisModel = SynthesisModel.load(model_name)

In [6]:
synthesis_model.construct_model()
synthesis_model.save()

start construct model


k = 12
reduced_prop = (0.7502298789712506+0j)
finish construct model, cost 1246.537829875946s


In [7]:
synthesis_model: SynthesisModel = SynthesisModel.load(model_name)

In [8]:
# TODO: 一个model的放到一个文件夹
circuits = gen_random_circuits(min_gate=500, max_gate=501, gate_num_step=1, n_circuits=1, two_qubit_gate_probs=[
                               3, 4], backend=backend, reverse=False, optimize=True, multi_process=False)[0]

qiskit_circuit = layered_circuits_to_qiskit(
    n_qubits, circuits['layer2gates'], barrier=False)
print(qiskit_circuit)


finish random circuit generation with 1 circuits
         ┌───────────────────┐        ┌────────────────────┐   »
q_0: ────┤ U(π/10,3π/5,4π/5) ├──────■─┤ U(3π/10,4π/5,2π/5) ├───»
         ├───────────────────┴┐     │ └─┬────────────────┬─┘   »
q_1: ────┤ U(3π/5,9π/10,π/10) ├─────■───┤ U(7π/10,π/5,π) ├───■─»
     ┌───┴────────────────────┴──┐      └────────────────┘   │ »
q_2: ┤ U(2.5432,-1.5595,-1.5935) ├──■────────────────────────■─»
     ├───────────────────────────┴┐ │                          »
q_3: ┤ U(2.0273,1.3266,-0.043998) ├─■──────────────────────────»
     └────────────────────────────┘                            »
«                                    ┌────────────────────────────┐      »
«q_0: ─────────────────────────────■─┤ U(0.88114,0.24134,0.86581) ├────■─»
«     ┌──────────────────────────┐ │ └┬──────────────────────────┬┘    │ »
«q_1: ┤ U(1.1343,1.0045,-1.5344) ├─■──┤ U(1.627,-1.3797,-2.8901) ├──■──■─»
«     └──┬────────────────────┬──┘    └──┬────────────────────┬──┘

In [9]:
init_unitary_mat_2 = qml.matrix(layer_circuit_to_pennylane_tape(circuits['layer2gates']))

In [10]:
init_unitary_mat = layer_circuit_to_matrix(circuits['layer2gates'], n_qubits)

In [11]:
gate_vectors = synthesis_model.data[1]
gate_vectors.shape

(1297140, 9)

In [12]:
Us = synthesis_model.data[0]
Us.shape, init_unitary_mat.shape

((1297140, 16, 16), (16, 16))

In [13]:
init_unitary_mat = Us[0]
from jax import vmap
import jax.numpy as jnp
dists = vmap(matrix_distance_squared, in_axes=(None, 0))(init_unitary_mat, Us)
max_index= np.argmin(dists)
dists[max_index]

Array(0.60447151, dtype=float64)

In [14]:
matrix_distance_squared(init_unitary_mat_2 @ init_unitary_mat_2.T.conj(), np.eye(16))

Array(6.88338275e-15, dtype=float64)

In [15]:
Us[0] @ Us[0].T.conj()

array([[ 0.60037623,  0.03504068,  0.09048404,  0.07480018, -0.16308691,
         0.01374751,  0.26221368,  0.05556214,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.03504068,  0.49909371,  0.08307675, -0.14627025, -0.14389167,
         0.11321141, -0.02700428, -0.02715377,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.09048404,  0.08307675,  0.25633293, -0.04434204, -0.09256992,
        -0.14634242, -0.15699984,  0.07428503,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.07480018, -0.14627025, -0.04434204,  0.16794286, -0.14656328,
        -0.05161886,  0.04117531, -0.00681115,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [-0.16308691, -0.14389167, -0

In [16]:
np.max(dists)

Array(1., dtype=float64)

In [17]:
device_gate_vector = gate_vectors[max_index]
upstream_model = synthesis_model.upstream_model
upstream_model.reconstruct(device_gate_vector[0], device_gate_vector[:])
device_gate_vector
upstream_model.extract_paths_from_vec(device_gate_vector[0], device_gate_vector[:])

['parallel-u,2', 'next-cz,1,2', 'parallel-cz,1,2']

In [18]:
device_gate_vec = synthesis_model.pca_model.transform(jnp.array([init_unitary_mat]))
device_gate_vec = synthesis_model.U_to_vec_model.predict(device_gate_vec)[0]
upstream_model.reconstruct(device_gate_vector[0], device_gate_vector[:])

[[{'name': 'u',
   'qubits': [2],
   'params': array([6.28318531, 6.28318531, 6.28318531])},
  {'name': 'cz', 'qubits': [1, 2], 'params': []}],
 [{'name': 'cz', 'qubits': [1, 2], 'params': []}]]

In [19]:

# params, dist = synthesis_model.find_parmas(n_qubits, circuits['layer2gates'], init_unitary_mat, max_epoch=50, allowed_dist=1e-2,
#                                 n_iter_no_change=5, no_change_tolerance=1e-3)

# init_unitary_mat = unitary_group.rvs(2**n_qubits)
start_time = time.time()
layer2gates = synthesis_model.synthesize(init_unitary_mat)
print('Synthesis costs', time.time() - start_time, 's')

start synthesis


MatrixUndefinedError: 